<a href="https://colab.research.google.com/github/100jy/EDA/blob/master/%EB%B9%85%EC%BD%98_simple_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
#구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
cd "/content/drive/My Drive/bigcon"

/content/drive/My Drive/bigcon


In [ ]:
!unzip -q data.zip -d /data

In [128]:
cd '/data'

/data


In [7]:
ls

'01_유동인구데이터(SK텔레콤)'/  '04_유통데이터(GS리테일)'/
'02_카드매출데이터(신한카드)'/  '05_물류데이터(CJ올리브네트웍스)'/
'03_SNS데이터(와이즈넛)'/


In [92]:
import pandas as pd
import numpy as np
from glob import glob

# 유동인구 데이터

## 19년도

In [40]:
nine_time = glob('./01_유동인구데이터(SK텔레콤)/*TIME*2019*.CSV')
nine_time.sort()

nine_age = glob('./01_유동인구데이터(SK텔레콤)/*AGE*2019*.CSV')
nine_age.sort()

# 19년도 날짜순으로 V-stacking
nine_time_merge = pd.read_csv(nine_time[0],sep = '|')
for i in  nine_time[1:]:
  new = pd.read_csv(i,sep = '|')
  nine_time_merge = pd.concat([nine_time_merge,new])

nine_age_merge = pd.read_csv(nine_age[0],sep = '|')
for i in  nine_age[1:]:
  new = pd.read_csv(i,sep = '|')
  nine_age_merge = pd.concat([nine_age_merge,new])

## 같은 날짜 같은 지역 H-stacking 
nine_merged = pd.merge(nine_time_merge,nine_age_merge, on = ['HDONG_NM','STD_YMD'])

nine_merged.head()
#print(nine_merged.shape,nine_time_merge.shape,nine_age_merge.shape)
nine_merged.to_csv("/content/drive/My Drive/bigcon/nine_merged.csv",encoding='euc-kr')

## 20년도

In [90]:
two_time = glob('./01_유동인구데이터(SK텔레콤)/*TIME*2020*.CSV')
two_time.sort()

two_age = glob('./01_유동인구데이터(SK텔레콤)/*AGE*2020*.CSV')
two_age.sort()

# 20년도 날짜순으로 V-stacking
two_time_merge = pd.read_csv(two_time[0],sep = '|')
for i in  two_time[1:]:
  new = pd.read_csv(i,sep = '|')
  two_time_merge = pd.concat([two_time_merge,new])

two_age_merge = pd.read_csv(two_age[0],sep = '|')
for i in  two_age[1:]:
  new = pd.read_csv(i,sep = '|')
  two_age_merge = pd.concat([two_age_merge,new])

## 같은 날짜 같은 지역 H-stacking 
two_merged = pd.merge(two_time_merge,two_age_merge, on = ['HDONG_NM','STD_YMD'])
two_merged.to_csv("/content/drive/My Drive/bigcon/two_merged.csv",encoding='euc-kr')

# 유통데이터

In [94]:
retail =  pd.read_excel('/content/drive/My Drive/bigcon/04_혁신아이디어분야_유통데이터(GS리테일)_데이터정의서 및 문제 데이터 1.xlsx')
retail.shape

(14460, 13)

In [113]:
#2019년도
#120일치 60개 지역
nine_retail = retail.iloc[:120*60]
#121일치 60개 지역
two_retail = retail.iloc[120*60:]

## 두 데이터 병합 

In [125]:
nine_pop = pd.read_csv("/content/drive/My Drive/bigcon/nine_merged.csv",encoding='euc-kr',index_col=0)
two_pop = pd.read_csv("/content/drive/My Drive/bigcon/two_merged.csv",encoding='euc-kr',index_col=0)
#120일치 69개 지역,121일치 69개 지역 
nine_pop.shape,two_pop.shape
#60개 지역, 120일치에 대해서만 join
nine_pop_retail = pd.merge(nine_pop,nine_retail, left_on =['HDONG_NM','STD_YMD'],right_on=['ADMD','OPER_DT'])
nine_pop_retail.to_csv("/content/drive/My Drive/bigcon/nine_pop_retail.csv")

In [146]:
#60개 지역, 121일치에 대해서만 join
two_pop_retail = pd.merge(two_pop,two_retail, left_on =['HDONG_NM','STD_YMD'],right_on=['ADMD','OPER_DT'])
two_pop_retail.to_csv("/content/drive/My Drive/bigcon/two_pop_retail.csv")

# 카드 매출 데이터

In [140]:
resident = glob('./02_카드매출데이터(신한카드)/CARD_SPENDING_RESIDENT/*')
foreigner = glob('./02_카드매출데이터(신한카드)/CARD_SPENDING_FOREIGNER/*')

resident_card = pd.read_csv(resident[0],sep = '	',encoding='euc-kr')
foreigner_card = pd.read_csv(foreigner[0],sep = '	',encoding='euc-kr')

In [175]:
resident_card
# 모든 변수 일별로,동별로 집계해주면 될 듯...
# 평균연령, 남여비, median cnt, median amt, MCT_CAT_CD
resident_card.groupby([resident_card['STD_DD'],resident_card['DONG_CD']]).median()

def get_sex_ratio(x):
  #남/여
  result = sum(x=='M')/sum(x=='F')
  return result

def get_cat(x):
  #최빈값
  result = x.value_counts().index[0]
  return result

resident_card_time_domain = resident_card.groupby([resident_card['STD_DD'],resident_card['DONG_CD']]).agg(
                                                  {'SEX_CD' : get_sex_ratio, 
                                                  'MCT_CAT_CD' :get_cat,
                                                  'AGE_CD' : np.median,
                                                  'USE_CNT':np.median,
                                                  'USE_AMT': np.median})
resident_card_time_domain = resident_card_time_domain.reset_index()

In [231]:
# 동이름 string으로 디코딩
name = pd.read_excel('/content/drive/My Drive/bigcon/행정동코드.xlsx')
name_dict ={}
for i,j in zip(name['행정동코드'],name['행정동명']):
  name_dict[i] = j
resident_card_time_domain[['DONG_CD']] = resident_card_time_domain[['DONG_CD']].applymap(name_dict.get)

In [236]:
#19년도
#47개 지역,120일
nine_card = resident_card_time_domain[:120*47]

#20년도
#47개지역, 121일
two_card = resident_card_time_domain[120*47:]

## 통합

In [308]:
#38개 지역,120일에 대해서
nine_pop_retail_card = pd.merge(nine_pop_retail,nine_card, left_on =['HDONG_NM','STD_YMD'],right_on=['DONG_CD','STD_DD'])
nine_pop_retail_card.to_csv("/content/drive/My Drive/bigcon/nine_pop_retail_card.csv")

#38개 지역,121일에 대해서
two_pop_retail_card = pd.merge(two_pop_retail,two_card, left_on =['HDONG_NM','STD_YMD'],right_on=['DONG_CD','STD_DD'])
two_pop_retail_card.to_csv("/content/drive/My Drive/bigcon/two_pop_retail_card.csv")

In [255]:
#서울 15개 지역
#대구 23개 지역
nine_pop_retail_card['PVN'].value_counts()/120

대구광역시    23.0
서울특별시    15.0
Name: PVN, dtype: float64

# 물류 데이터

In [262]:
logistics = pd.read_excel('./05_물류데이터(CJ올리브네트웍스)/2020 빅콘테스트_CJ올리브네트웍스_제공DB.xlsx')

In [305]:
logistics_reduce = logistics[['DL_YMD','DL_GD_LCLS_NM','HDNG_NM','INVC_CONT']]
#코드는 다버리고, 동이름만 남기고, 물건종류는 더미변수로 해서 sum으로 reduce,송장건수도 sum
logistics_reduce = logistics_reduce.join(pd.get_dummies(logistics_reduce['DL_GD_LCLS_NM']))
del logistics_reduce['DL_GD_LCLS_NM']
logistics_reduce = logistics_reduce.groupby([logistics_reduce['DL_YMD'],logistics_reduce['HDNG_NM']]).sum()
del logistics_reduce['DL_YMD']
logistics_reduce = logistics_reduce.reset_index()
#날짜 인코딩 형식 맞춰주기
logistics_reduce['DL_YMD'] = [int('20' + str(x)) for x in logistics_reduce['DL_YMD']]

In [309]:
#19년도
# 일자별로 해당지역에 물류이동이 없는 경우 발생..
nine_without_sns = pd.merge(nine_pop_retail_card,logistics_reduce, left_on =['HDONG_NM','STD_YMD'],right_on=['HDNG_NM','DL_YMD'])
#20년도
two_without_sns = pd.merge(two_pop_retail_card,logistics_reduce, left_on =['HDONG_NM','STD_YMD'],right_on=['HDNG_NM','DL_YMD'])

In [ ]:
nine_pop_retail_card.to_csv("/content/drive/My Drive/bigcon/nine_pop_retail_card.csv")

1. 지역별 특성값을 일별로 산출하기
  - 유동인구 + sns + 물류 +유통
  - 타임인덱스 기준으로 조인하기..
 

2. 업종별 특성값을 일별로 산출하기
  - 나머지 자료를 이용하여...